# Metaprogramming

A "meta" program is a program that manipulates programs.

Most common use refers to a program that *generates* another program.

In the last class we saw how Julia can generate specialized code for you.

Sometimes that's not enough, and you need to write a program to explicitly generate the code needed for a specialized problem.

Julia allows us to talk in a "meta" way ("one level up"), about Julia code,  that is to "treat code as data" and manipulate it as just another object in Julia. (This is very similar to Lisp.)

## Symbols and Expressions

One of the most basic objects in this approach are unevaluated *symbols*:

In [1]:
:a  # "the symbol a"

:a

In [2]:
typeof(:a)

Symbol

`:a` refers to the symbol `a`. We can evaluate it with the `eval` function:

In [3]:
eval(:a)

UndefVarError: UndefVarError: a not defined

`a` must be defined for this to work:

In [4]:
a = 3

3

In [5]:
eval(:a)

3

In [6]:
a

3

In [7]:
dump(a)

Int64 3


The `eval` function takes an expression and evaluates it, that is, *generates the corresponding code*

Operators and function names are also symbols:

In [8]:
:+, :sin

(:+, :sin)

In [9]:
typeof(:+)

Symbol

Symbols may be combined into *expressions*, which are the basic objects that represent pieces of Julia code:

In [10]:
ex = :(a + b)  # the expression 'a + b'

:(a + b)

In [11]:
typeof(ex)

Expr

In [12]:
eval(ex)

UndefVarError: UndefVarError: b not defined

In [13]:
b = 7
eval(ex)

10

In [14]:
a

3

In [15]:
ex

:(a + b)

An expression is just a Julia object, so we can introspect (find out information about it):

In [16]:
dump(ex)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Symbol a
    3: Symbol b


In [17]:
dump(:(x = 3))

Expr
  head: Symbol =
  args: Array{Any}((2,))
    1: Symbol x
    2: Int64 3


In [ ]:
# ex.<TAB>

In [18]:
ex.head

:call

In [19]:
ex.args

3-element Array{Any,1}:
 :+
 :a
 :b

The job of Julia's parser is to convert a sequence of characters into these `Expr` objects:

In [20]:
parse("a + b")

MethodError: MethodError: no method matching parse(::String)
Closest candidates are:
  parse(!Matched::Type{T}, !Matched::AbstractChar; base) where T<:Integer at parse.jl:41
  parse(!Matched::Type{T}, !Matched::AbstractString; base) where T<:Integer at parse.jl:238
  parse(!Matched::Type{T}, !Matched::AbstractString; kwargs...) where T<:Real at parse.jl:376
  ...

More complicated expressions are represented as "abstract syntax trees" (ASTs), consisting of expressions nested inside expressions:

In [21]:
ex = :( sin(3a + 2b^2) )

:(sin(3a + 2 * b ^ 2))

In [22]:
ex.args

2-element Array{Any,1}:
 :sin
 :(3a + 2 * b ^ 2)

In [23]:
typeof(ex.args[2])

Expr

In [24]:
ex.args[2].args

3-element Array{Any,1}:
 :+
 :(3a)
 :(2 * b ^ 2)

In [25]:
ex.args[1] = :cos

:cos

In [26]:
ex

:(cos(3a + 2 * b ^ 2))

In [27]:
blk = quote
    println("Hello")
end

quote
    #= In[27]:2 =#
    println("Hello")
end

In [28]:
eval(blk)

Hello


In [29]:
push!(blk.args, :(println("AFTER")))

3-element Array{Any,1}:
 :(#= In[27]:2 =#)
 :(println("Hello"))
 :(println("AFTER"))

In [30]:
blk

quote
    #= In[27]:2 =#
    println("Hello")
    println("AFTER")
end

In [31]:
eval(blk)

Hello
AFTER


In [37]:
?lpush

search:

Couldn't find lpush
Perhaps you meant flush, push!, lpad, less, last, ltoh, acosh, cosh, eps or hash


No documentation found.

Binding `lpush` does not exist.


In [36]:
unshift!(blk.args, :(println("BEFORE")))

UndefVarError: UndefVarError: lpush! not defined

In [33]:
eval(blk)

Hello
AFTER


Expressions can be arbitrary Julia code that when evaluated will have side effects. For longer blocks of code, `quote...end` may be used instead of `:( ... )`

In [38]:
ex2 = 
quote
    y = 3
    z = sin(y+1)
end

quote
    #= In[38]:3 =#
    y = 3
    #= In[38]:4 =#
    z = sin(y + 1)
end

In [39]:
eval(ex2)
y

3

In [42]:
eval(ex2)
z

-0.7568024953079282

The full form of the abstract syntax tree in a style similar to a Lisp s-expression can be obtained using functions from the `Meta` module in `Base`:

In [43]:
Meta.show_sexpr(ex2)

(:block,
  :(#= In[38]:3 =#),
  (:(=), :y, 3),
  :(#= In[38]:4 =#),
  (:(=), :z, (:call, :sin, (:call, :+, :y, 1)))
)

Another way of seeing the structure is with `dump`:

In [44]:
dump(ex2)

Expr
  head: Symbol block
  args: Array{Any}((4,))
    1: LineNumberNode
      line: Int64 3
      file: Symbol In[38]
    2: Expr
      head: Symbol =
      args: Array{Any}((2,))
        1: Symbol y
        2: Int64 3
    3: LineNumberNode
      line: Int64 4
      file: Symbol In[38]
    4: Expr
      head: Symbol =
      args: Array{Any}((2,))
        1: Symbol z
        2: Expr
          head: Symbol call
          args: Array{Any}((2,))
            1: Symbol sin
            2: Expr
              head: Symbol call
              args: Array{Any}((3,))
                1: Symbol +
                2: Symbol y
                3: Int64 1


## Interpolation

Values can be *interpolated* into expressions with `$`.

In [45]:
x = 2

2

In [46]:
:(x + $x)

:(x + 2)

In [47]:
x = :y

:y

In [48]:
:(x + $x)

:(x + y)

### Why not strings?

*Abstract* syntax vs. *concrete* syntax.

In [49]:
a = "x + y"

"x + y"

In [50]:
b = "z"

"z"

In [51]:
expr = "$a * z"

"x + y * z"

Even worse, what if `z == ")"`?

With `Expr` objects, every expression is well-formed and each slot has its abstract meaning.

### Splat interpolation

In [52]:
vars = [:a, :b, :c]

3-element Array{Symbol,1}:
 :a
 :b
 :c

In [53]:
:(1 - f(0, $(vars...)))

:(1 - f(0, a, b, c))

### `@eval` for interpolating on the fly

You will sometimes see `@eval expr`, which is shorthand for:

```
eval(quote
         expr
     end)
```

allowing items to be interpolated immediately before executing.

In [54]:
fnames = [ Symbol("func$i") for i=1:2]

2-element Array{Symbol,1}:
 :func1
 :func2

In [55]:
for i = 1:length(fnames)
    @eval ($(fnames[i]))(x) = $i
end

In [56]:
func1(0)

1

In [57]:
func2(0)

2

## Macros

With the ability to think of code in terms of a data structure in the Julia language, we can now *manipulate* those data structures, allowing us to *create Julia code from within Julia*.

*Macros* provide a particular use pattern of metaprogramming: replacing one expression with another, in-place, right after parsing.

The [Julia manual](http://julia.readthedocs.org/en/latest/manual/metaprogramming/) puts it like this: 

> macros map a tuple of argument *expressions* to a returned
> *expression*

Macros are useful in several cases:

- to provide a specific notation different than what can normally be written in the language
    - e.g. https://github.com/JuliaOpt/JuMP.jl/blob/release-0.18/examples/sudoku.jl#L44
- to rearrange or delay evaluation of code
- to eliminate boilerplate (repetitive) code
- to automatically generate complex code that would be painful by hand
- to unroll loops for efficiency
- to inline code for efficiency



Macros are invoked using the `@` sign, e.g.

In [58]:
@time sin(10)

  0.000000 seconds


-0.5440211108893698

A trivial example of defining a macro is the following, which runs whatever code it is passed two times.

In [59]:
macro twice(ex)
    quote
        $ex
        $ex
    end
end

@twice (macro with 1 method)

In [60]:
x = 0
@twice println(x += 1)

ErrorException: syntax: invalid assignment location "Main.x"

In [61]:
ex = :(@twice println(x += 1))

:(#= In[61]:1 =# @twice println(x += 1))

In [62]:
eval(ex)

ErrorException: syntax: invalid assignment location "Main.x"

In [63]:
typeof(ex)

Expr

We can see what effect the macro actually has using `macroexpand`:

In [64]:
macroexpand(:(@twice println(x += 1)))

MethodError: MethodError: no method matching macroexpand(::Expr)
Closest candidates are:
  macroexpand(!Matched::Module, !Matched::Any; recursive) at expr.jl:107

In [65]:
macroexpand(:(@time sin(10)))

MethodError: MethodError: no method matching macroexpand(::Expr)
Closest candidates are:
  macroexpand(!Matched::Module, !Matched::Any; recursive) at expr.jl:107

In [66]:
macro mytime(ex)
    quote
        t0 = time()
        val = $ex
        t1 = time()
        println("$(t1-t0) seconds elapsed")
        val
    end
end

@mytime (macro with 1 method)

In [67]:
@mytime (sleep(1); "done")

1.0130000114440918 seconds elapsed


"done"

A line break terminates macro arguments, so blocks need to be passed with `begin...end`:

In [68]:
@time begin
    # code
end

  0.000000 seconds


----
**Exercise**: Define a macro `@until` that does an `until` loop.

----

### Macro Hygiene

In [ ]:
macro set_x(val)
    :(x = $val)
end

In [ ]:
@set_x 10

In [ ]:
x

In [ ]:
macro set_x(val)
    :($(esc(:x)) = $val)
end

In [ ]:
@set_x 10

In [ ]:
x

### Macros for numerical performance: Horner's method

There are many interesting examples of macros in `Base`. One that is accessible is Horner's method for evaluating a polynomial:

$$p(x) = a_n x^n + a_{n-1} x^{n-1} + \cdots + a_1 x^1 + a_0$$

may be evaluated efficiently as

$$p(x) = a_0 + x(a_1 + \cdots x(a_{n-2} + \cdots + x(a_{n-1} + x a_n) \cdots ) ) $$
with only $n$ multiplications.

The obvious way to do this is with a `for` loop. But if we know the polynomial *at compile time*, this loop may be unrolled using metaprogramming. This is implemented in the `Math` module in `math.jl` in `Base`, so the name of the macro (which is not exported) is `@Base.Math.horner`.

In [ ]:
# copied from base/math.jl
macro horner(x, p...)
    ex = esc(p[end])
    for i = length(p)-1:-1:1
        ex = :( $(esc(p[i])) + t * $ex )  # actually uses `muladd`
    end
    Expr(:block, :(t = $(esc(x))), ex)
end

This is called as follows: to evaluate the polynomial $p(x) = 2 + 3x + 4x^2$ at $x=3$, we do

In [ ]:
x = 3
@horner(x, 2, 3, 4, 5)

To see what the macro does to this call, we again use `macroexpand`:

In [ ]:
macroexpand(:(@horner(x, 2, 3, 4, 5, 6, 7, 8, 9, 10)))

In [ ]:
macroexpand(:(@Base.Math.horner(x, 2, 3, 4, 5)))

In [ ]:
?muladd

In [ ]:
?fma

In [ ]:
f(x) = Base.Math.@horner(x, 1.2, 2.3, 3.4, 4.5)

In [ ]:
@code_llvm f(0.1)

In [ ]:
@code_native f(0.1)

## `@generated` Functions

Another form of metaprogramming available in Julia.

Generate code based on *types* instead of the input expressions.

In [ ]:
@generated function nloops(a::Array{T,N}) where {T,N}
    # inside here,
    # - `a` refers to the type of the argument
    # - `T` and `N` have their usual values
    # - return an expression to compute the answer for this type
end